In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

from google.colab import drive
drive.mount('/content/drive')

import random
import cv2
from matplotlib import pyplot as plt
import albumentations as A
import pandas as pd
import os
import json

Mounted at /gdrive
/gdrive
Mounted at /content/drive


In [56]:
#------------------ORIGINAL------------------------
# TRAIN_JSON = "/content/drive/My Drive/thyroid_nodule_detection/Experiment_3/dataset/load_json/train.json"
# TRAIN_DIR = "/content/drive/My Drive/thyroid_nodule_detection/dataset/train/"

# VALID_JSON = "/content/drive/My Drive/thyroid_nodule_detection/Experiment_3/dataset/load_json/validation.json"
# VALID_DIR = "/content/drive/My Drive/thyroid_nodule_detection/dataset/validation/"

# RESULT_JSON_FILE = "/content/drive/My Drive/thyroid_nodule_detection/Experiment_3/dataset/validation/resized_images.json"

# savepath = "/content/drive/My Drive/thyroid_nodule_detection/Experiment_3/dataset/validation/resized_images/"
#------------------ORIGINAL------------------------

#------------------FIXED LABELS------------------------
TRAIN_JSON = "/content/drive/My Drive/thyroid_nodule_detection/Freehand to Bounding Box Conversion Fix/fixed_labels/train.json"
TRAIN_DIR = "/content/drive/My Drive/thyroid_nodule_detection/dataset/train/"

VALID_JSON = "/content/drive/My Drive/thyroid_nodule_detection/Freehand to Bounding Box Conversion Fix/fixed_labels/validation.json"
VALID_DIR = "/content/drive/My Drive/thyroid_nodule_detection/dataset/validation/"


# Change paths to train/validation accordingly
RESULT_JSON_FILE = "/content/drive/My Drive/thyroid_nodule_detection/Freehand to Bounding Box Conversion Fix/fixed_labels/cropped_dataset/validation/cropped_labels.json"
savepath = "/content/drive/My Drive/thyroid_nodule_detection/Freehand to Bounding Box Conversion Fix/fixed_labels/cropped_dataset/validation/cropped_images/"
#------------------FIXED LABELS------------------------

In [3]:
temp_train_df = pd.read_json(TRAIN_JSON)

In [4]:
temp_valid_df = pd.read_json(VALID_JSON)

In [5]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)
    # x_min, y_min, x_max, y_max = bbox
    # x_min, x_max, y_min, y_max = int(x_min), int(x_max), int(y_min), int(y_max)
   
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return img

def visualize(image, bboxes, category_ids, category_id_to_name):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

In [48]:
transform = A.Compose(
    [A.CenterCrop(height=350, width=400, p=1)],
    bbox_params=A.BboxParams(format='coco', min_area=1, label_fields=['category_ids']),
)

In [57]:
def process_df(df):
    cases = df
    case_list = []
    for case in df['cases']:
        image_id = case['case_id'] + ".jpg"
        temp_case = {}
        
        temp_case['case_id'] = case['case_id']
        temp_case['image'] = cv2.imread(os.path.join(VALID_DIR, image_id), cv2.IMREAD_COLOR) # Change path to train/validation accordingly
        temp_case['tirads'] = case['tirads']
        temp_case['bboxes'] = case['bboxes']
        case_list.append(temp_case)
    return pd.DataFrame(case_list)

df_general = process_df(temp_valid_df) # Change df to train/validation accordingly

In [8]:
category_id_to_name = {2: 'benign', 3: 'benign',4: 'malignant',5: 'malignant' }

In [49]:
def proccess_bboxes(bboxes):
  final_bboxes = []
  for bbox in bboxes:
    # temp_bbox = [
    #     bbox['xmin'],
    #     bbox['ymin'],
    #     bbox['xmax'],
    #     bbox['ymax'],
    # ]

    temp_bbox = [
        bbox['x'],
        bbox['y'],
        bbox['w'],
        bbox['h'],
    ]
    final_bboxes.append(temp_bbox)
  
  return final_bboxes

In [50]:
def create_transormed_images(df):
  transformed_array = []
  for index,row in df.iterrows():
    id = row.get('tirads')
    if len(id) == 2:
      id = 4
    else:
      id = int(id)

    
    lenght_of_array = len(row.get('bboxes'))
    ## To add [id], lenght_of_array times, to category_ids:
    category_ids = lenght_of_array * [id]
    image = row.get('image')
    bboxes = proccess_bboxes(row.get('bboxes'))
    id = row.get('case_id')
    temp_transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)
    transormed_data = {
          'case_id': row.get('case_id'),
          'image': temp_transformed['image'],
          'bboxes': temp_transformed['bboxes'],
          'labels': temp_transformed['category_ids'],
      }
    transformed_array.append(transormed_data)

    cases = {
        'cases':transformed_array
    }
  return cases


usr/local/lib/python3.7/dist-packages/albumentations/core/bbox_utils.py/check_bbox

```
  bbox=list(bbox)
    for i in range(4):
      if bbox[i]<0:
        bbox[i]=0
      elif bbox[i]>1:
        bbox[i]=1
    bbox=tuple(bbox)
```


command + m + . = restart runtime

In [58]:
transformed_data = create_transormed_images(df_general)
# print(transformed_data)

399_1
[{'x': 254, 'y': 110, 'w': 50, 'h': 59}, {'x': 275, 'y': 156, 'w': 60, 'h': 106}]
[[254, 110, 50, 59], [275, 156, 60, 106]]
398_1
[{'x': 264, 'y': 85, 'w': 92, 'h': 131}]
[[264, 85, 92, 131]]
400_1
[{'x': 207, 'y': 84, 'w': 170, 'h': 147}]
[[207, 84, 170, 147]]
392_1
[{'x': 249, 'y': 75, 'w': 56, 'h': 95}]
[[249, 75, 56, 95]]
394_1
[{'x': 146, 'y': 34, 'w': 264, 'h': 194}]
[[146, 34, 264, 194]]
391_1
[{'x': 220, 'y': 114, 'w': 155, 'h': 75}]
[[220, 114, 155, 75]]
391_2
[{'x': 235, 'y': 86, 'w': 144, 'h': 111}]
[[235, 86, 144, 111]]
397_1
[{'x': 297, 'y': 127, 'w': 99, 'h': 81}]
[[297, 127, 99, 81]]
395_2
[{'x': 260, 'y': 62, 'w': 88, 'h': 129}]
[[260, 62, 88, 129]]
395_1
[{'x': 202, 'y': 52, 'w': 163, 'h': 103}]
[[202, 52, 163, 103]]


**SAVE IMAGES**

In [59]:
import io
import PIL.Image as Image

for d in transformed_data['cases']:
  if len(d['bboxes']) >= 1:
    image_path = savepath + d['case_id'] + '.jpg'
    im = Image.fromarray(d['image'])
    im.save(image_path)

In [60]:
temp_cases = []
for case in transformed_data['cases']:
  if len(case['bboxes']) >= 1:
    case = {
        'case_id':case['case_id'],
        'bboxes': case['bboxes'],
        'labels': case['labels']
    }
    temp_cases.append(case)

cases = {
    'cases': temp_cases
}
print(cases)

text_file = open(RESULT_JSON_FILE, "w")
text_file.write(json.dumps(cases))
text_file.close()

    

{'cases': [{'case_id': '399_1', 'bboxes': [(174.0, 105.00000000000001, 50.00000000000003, 58.999999999999986), (195.0, 151.0, 59.99999999999997, 106.0)], 'labels': [4, 4]}, {'case_id': '398_1', 'bboxes': [(184.0, 80.0, 92.0, 131.0)], 'labels': [4]}, {'case_id': '400_1', 'bboxes': [(127.0, 79.0, 170.0, 147.00000000000003)], 'labels': [4]}, {'case_id': '392_1', 'bboxes': [(169.0, 70.0, 56.0, 95.0)], 'labels': [4]}, {'case_id': '394_1', 'bboxes': [(66.0, 28.999999999999996, 264.0, 194.0)], 'labels': [5]}, {'case_id': '391_1', 'bboxes': [(140.0, 109.0, 155.0, 75.0)], 'labels': [2]}, {'case_id': '391_2', 'bboxes': [(155.0, 81.0, 144.0, 111.0)], 'labels': [2]}, {'case_id': '397_1', 'bboxes': [(217.0, 122.0, 99.0, 81.0)], 'labels': [2]}, {'case_id': '395_2', 'bboxes': [(180.0, 57.0, 88.0, 129.00000000000003)], 'labels': [4]}, {'case_id': '395_1', 'bboxes': [(122.0, 46.99999999999999, 163.0, 103.0)], 'labels': [4]}]}


In [ ]:
# def proccess_transformed_data_array(transformed_data):
#   array = []
#   for transformed in transformed_data:
#     if len(transformed['bboxes']) >= 1:
#       temp_data = {}
#       img = transformed['image'].copy()
#       plt.imshow(img)
#     array.append(temp_data)

#   numpyData = {"array": array}
#   return numpyData

# serilizable_list = proccess_transformed_data_array(transformed_data)


# def save_json(data):
#     text_file = open(JSON_FILE, "w")
#     text_file.write(data)
#     text_file.close()

# save_json(str(serilizable_list))

# for transformed in transformed_data:
#   if len(transformed['bboxes']) >= 1:

#     print(transformed['bboxes'])
#      visualize(
#        transformed['image'],
#        transformed['bboxes'],
#        transformed['category_ids'],
#        category_id_to_name)